In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13939,2021-12-18T17:00:00,ITA,19,Sicilia,38.115697,13.362357,494,55,549,19326,...,DECEDUTI: N. 3 IL 17/12/21 - N. 5 IL 16/12/21 ...,9.0,NaN,NaN,341285.0,0.0,3674956.0,4233164.0,ITG,ITG1
13940,2021-12-18T17:00:00,ITA,9,Toscana,43.769231,11.255889,331,58,389,15500,...,NaN,6.0,NaN,NaN,309988.0,5986.0,5225978.0,3318578.0,ITI,ITI1
13941,2021-12-18T17:00:00,ITA,10,Umbria,43.106758,12.388247,52,8,60,3366,...,"Si fa presente che due dei ricoveri NON UTI, n...",0.0,NaN,NaN,70819.0,0.0,1291716.0,1371892.0,ITI,ITI2
13942,2021-12-18T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,20,2,22,851,...,NaN,0.0,NaN,NaN,12700.0,1535.0,121769.0,204910.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13939,2021-12-18T17:00:00,19,Sicilia,494,55,549,19326,19875,757,1368,...,341285,7908120,2870067.0,9.0,341285.0,0.0,3674956.0,4233164.0,ITG,ITG1
13940,2021-12-18T17:00:00,9,Toscana,331,58,389,15500,15889,809,1282,...,315974,8544556,3484986.0,6.0,309988.0,5986.0,5225978.0,3318578.0,ITI,ITI1
13941,2021-12-18T17:00:00,10,Umbria,52,8,60,3366,3426,272,372,...,70819,2663608,484817.0,0.0,70819.0,0.0,1291716.0,1371892.0,ITI,ITI2
13942,2021-12-18T17:00:00,2,Valle d'Aosta,20,2,22,851,873,59,92,...,14235,326679,102753.0,0.0,12700.0,1535.0,121769.0,204910.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-12-18', '2021-12-17')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-12-18T17:00:00,1,Piemonte,664,56,720,23897,24617,1334,2326,...,425688,10460225,2879801.0,7.0,382375.0,43313.0,4003785.0,6456440.0,ITC,ITC1
1,2021-12-18T17:00:00,2,Valle d'Aosta,20,2,22,851,873,59,92,...,14235,326679,102753.0,0.0,12700.0,1535.0,121769.0,204910.0,ITC,ITC2
2,2021-12-18T17:00:00,3,Lombardia,1223,154,1377,61740,63117,4204,6119,...,992464,22757576,6432037.0,13.0,887122.0,105342.0,12467050.0,10290526.0,ITC,ITC4
3,2021-12-18T17:00:00,5,Veneto,974,153,1127,57278,58405,1367,4016,...,574858,18193806,2394268.0,20.0,547601.0,27257.0,7997026.0,10196780.0,ITH,ITH3
4,2021-12-18T17:00:00,6,Friuli Venezia Giulia,285,34,319,8002,8321,104,964,...,143346,4257995,936370.0,6.0,124892.0,18454.0,2553584.0,1704411.0,ITH,ITH4
5,2021-12-18T17:00:00,7,Liguria,337,31,368,7758,8126,570,1007,...,133551,3302283,1002018.0,3.0,133551.0,0.0,1909756.0,1392527.0,ITC,ITC3
6,2021-12-18T17:00:00,8,Emilia-Romagna,986,107,1093,38874,39967,1783,2451,...,487168,11129869,2327435.0,12.0,486698.0,470.0,6982963.0,4146906.0,ITH,ITH5
7,2021-12-18T17:00:00,9,Toscana,331,58,389,15500,15889,809,1282,...,315974,8544556,3484986.0,6.0,309988.0,5986.0,5225978.0,3318578.0,ITI,ITI1
8,2021-12-18T17:00:00,10,Umbria,52,8,60,3366,3426,272,372,...,70819,2663608,484817.0,0.0,70819.0,0.0,1291716.0,1371892.0,ITI,ITI2
9,2021-12-18T17:00:00,11,Marche,163,38,201,6316,6517,101,703,...,132734,1870451,1080994.0,3.0,132734.0,0.0,1570445.0,300006.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-12-17T17:00:00,1,Piemonte,667,53,720,22563,23283,1559,2510,...,423362,10393625,2869285.0,6.0,381307.0,42055.0,3991497.0,6402128.0,ITC,ITC1
1,2021-12-17T17:00:00,2,Valle d'Aosta,18,2,20,794,814,42,83,...,14143,324309,102323.0,0.0,12644.0,1499.0,121542.0,202767.0,ITC,ITC2
2,2021-12-17T17:00:00,3,Lombardia,1193,147,1340,57573,58913,2985,5590,...,986345,22607901,6407383.0,10.0,883911.0,102434.0,12428833.0,10179068.0,ITC,ITC4
3,2021-12-17T17:00:00,5,Veneto,980,154,1134,55904,57038,3100,5577,...,570842,18086398,2388697.0,14.0,544582.0,26260.0,7973376.0,10113022.0,ITH,ITH3
4,2021-12-17T17:00:00,6,Friuli Venezia Giulia,286,31,317,7900,8217,83,809,...,142386,4232567,934051.0,0.0,124038.0,18348.0,2543791.0,1688776.0,ITH,ITH4
5,2021-12-17T17:00:00,7,Liguria,337,29,366,7190,7556,208,1023,...,132544,3283675,999085.0,1.0,132544.0,0.0,1903331.0,1380344.0,ITC,ITC3
6,2021-12-17T17:00:00,8,Emilia-Romagna,989,102,1091,37093,38184,1058,2507,...,484719,11090651,2321983.0,5.0,484255.0,464.0,6959112.0,4131539.0,ITH,ITH5
7,2021-12-17T17:00:00,9,Toscana,335,55,390,14690,15080,680,1227,...,314692,8506932,3473593.0,4.0,308754.0,5938.0,5213951.0,3292981.0,ITI,ITI1
8,2021-12-17T17:00:00,10,Umbria,51,8,59,3095,3154,285,368,...,70447,2650236,483959.0,0.0,70447.0,0.0,1288467.0,1361769.0,ITI,ITI2
9,2021-12-17T17:00:00,11,Marche,155,36,191,6225,6416,26,682,...,132031,1862849,1076455.0,1.0,132031.0,0.0,1565313.0,297536.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-12-18T17:00:00,1,Piemonte,664,56,720,23897,24617,1334,2326,...,10460225,2879801.0,7.0,382375.0,43313.0,4003785.0,6456440.0,ITC,ITC1,4.341375e+06
1,2021-12-18T17:00:00,2,Valle d'Aosta,20,2,22,851,873,59,92,...,326679,102753.0,0.0,12700.0,1535.0,121769.0,204910.0,ITC,ITC2,1.255010e+05
2,2021-12-18T17:00:00,3,Lombardia,1223,154,1377,61740,63117,4204,6119,...,22757576,6432037.0,13.0,887122.0,105342.0,12467050.0,10290526.0,ITC,ITC4,1.010397e+07
3,2021-12-18T17:00:00,5,Veneto,974,153,1127,57278,58405,1367,4016,...,18193806,2394268.0,20.0,547601.0,27257.0,7997026.0,10196780.0,ITH,ITH3,4.907704e+06
4,2021-12-18T17:00:00,6,Friuli Venezia Giulia,285,34,319,8002,8321,104,964,...,4257995,936370.0,6.0,124892.0,18454.0,2553584.0,1704411.0,ITH,ITH4,1.211357e+06
5,2021-12-18T17:00:00,7,Liguria,337,31,368,7758,8126,570,1007,...,3302283,1002018.0,3.0,133551.0,0.0,1909756.0,1392527.0,ITC,ITC3,1.543127e+06
6,2021-12-18T17:00:00,8,Emilia-Romagna,986,107,1093,38874,39967,1783,2451,...,11129869,2327435.0,12.0,486698.0,470.0,6982963.0,4146906.0,ITH,ITH5,4.467118e+06
7,2021-12-18T17:00:00,9,Toscana,331,58,389,15500,15889,809,1282,...,8544556,3484986.0,6.0,309988.0,5986.0,5225978.0,3318578.0,ITI,ITI1,3.722729e+06
8,2021-12-18T17:00:00,10,Umbria,52,8,60,3366,3426,272,372,...,2663608,484817.0,0.0,70819.0,0.0,1291716.0,1371892.0,ITI,ITI2,8.802850e+05
9,2021-12-18T17:00:00,11,Marche,163,38,201,6316,6517,101,703,...,1870451,1080994.0,3.0,132734.0,0.0,1570445.0,300006.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-12-18T17:00:00,1,Piemonte,664,56,720,23897,24617,1334,2326,...,6456440.0,ITC,ITC1,4.341375e+06,Piemonte,667,5824,53,628,99
1,2021-12-18T17:00:00,2,Valle d'Aosta,20,2,22,851,873,59,92,...,204910.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,18,99,2,33,0
2,2021-12-18T17:00:00,3,Lombardia,1223,154,1377,61740,63117,4204,6119,...,10290526.0,ITC,ITC4,1.010397e+07,Lombardia,1193,8485,147,1530,0
3,2021-12-18T17:00:00,5,Veneto,974,153,1127,57278,58405,1367,4016,...,10196780.0,ITH,ITH3,4.907704e+06,Veneto,980,6000,154,1000,0
4,2021-12-18T17:00:00,6,Friuli Venezia Giulia,285,34,319,8002,8321,104,964,...,1704411.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,286,1277,31,175,0
5,2021-12-18T17:00:00,7,Liguria,337,31,368,7758,8126,570,1007,...,1392527.0,ITC,ITC3,1.543127e+06,Liguria,337,1756,29,219,12
6,2021-12-18T17:00:00,8,Emilia-Romagna,986,107,1093,38874,39967,1783,2451,...,4146906.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,989,9001,102,889,0
7,2021-12-18T17:00:00,9,Toscana,331,58,389,15500,15889,809,1282,...,3318578.0,ITI,ITI1,3.722729e+06,Toscana,335,5033,55,570,32
8,2021-12-18T17:00:00,10,Umbria,52,8,60,3366,3426,272,372,...,1371892.0,ITI,ITI2,8.802850e+05,Umbria,51,662,8,88,39
9,2021-12-18T17:00:00,11,Marche,163,38,201,6316,6517,101,703,...,300006.0,ITI,ITI3,1.518400e+06,Marche,155,970,36,250,52


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-12-18T17:00:00,3,Lombardia,1223,154,1377,61740,63117,4204,6119,...,10.07,4167,529,149675,24654.0,24,1891,4.09,0.06056,9.82
1,2021-12-18T17:00:00,5,Veneto,974,153,1127,57278,58405,1367,4016,...,15.30,1374,-1561,107408,5571.0,26,2623,3.74,0.08183,11.71
2,2021-12-18T17:00:00,8,Emilia-Romagna,986,107,1093,38874,39967,1783,2451,...,12.04,1781,-56,39218,5452.0,15,651,6.25,0.05487,10.91
3,2021-12-18T17:00:00,12,Lazio,832,112,944,35260,36204,1549,2409,...,11.88,1544,288,64142,2311.0,6,854,3.76,0.04107,7.76
4,2021-12-18T17:00:00,1,Piemonte,664,56,720,23897,24617,1334,2326,...,8.92,1334,-184,66600,10516.0,8,984,3.49,0.05358,9.81
5,2021-12-18T17:00:00,15,Campania,418,31,449,24756,25205,1214,2256,...,5.96,1210,415,43426,4931.0,6,1036,5.20,0.03899,8.90
6,2021-12-18T17:00:00,19,Sicilia,494,55,549,19326,19875,757,1368,...,6.37,741,-154,30836,9264.0,10,601,4.44,0.02753,6.87
7,2021-12-18T17:00:00,9,Toscana,331,58,389,15500,15889,809,1282,...,10.18,810,55,37624,11393.0,4,469,3.41,0.03444,8.49
8,2021-12-18T17:00:00,7,Liguria,337,31,368,7758,8126,570,1007,...,14.16,568,-16,18608,2933.0,4,433,5.41,0.06526,8.65
9,2021-12-18T17:00:00,6,Friuli Venezia Giulia,285,34,319,8002,8321,104,964,...,19.43,102,155,25428,2319.0,6,850,3.79,0.07958,11.83


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,P.A. Trento,21,69,90,23.33
1,Friuli Venezia Giulia,34,141,175,19.43
2,P.A. Bolzano,18,82,100,18.00
3,Veneto,153,847,1000,15.30
4,Marche,38,212,250,15.20
5,Liguria,31,188,219,14.16
6,Emilia-Romagna,107,782,889,12.04
7,Lazio,112,831,943,11.88
8,Calabria,19,161,180,10.56
9,Toscana,58,512,570,10.18


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))